In [ ]:
# For now, clear outputs (Cell > All Output > Clear) before committing to Git
# There might be a better way

import sqlite3
import pandas as pd
cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')

import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree, preprocessing
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_sql_query("SELECT FIRE_YEAR,DISCOVERY_TIME,STAT_CAUSE_DESCR,CONT_DATE,CONT_TIME,LATITUDE,LONGITUDE,STATE,DISCOVERY_DATE,FIRE_SIZE,FIRE_SIZE_CLASS FROM 'Fires'", cnx)
print(df.head())

In [ ]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
print(df.head())

In [ ]:
df['DISCOVERY_MONTH'] = pd.DatetimeIndex(df['DISCOVERY_DATE']).month
df['DISCOVERY_DAY'] = pd.DatetimeIndex(df['DISCOVERY_DATE']).day
df['DISCOVERY_DAY_OF_WEEK'] = df['DISCOVERY_DATE'].dt.weekday_name

#df['CONT_DATE'].fillna(0)
df['CONT_MONTH'] = pd.DatetimeIndex(df['CONT_DATE']).month
df['CONT_DAY'] = pd.DatetimeIndex(df['CONT_DATE']).day
df['CONT_DAY_OF_WEEK'] = df['CONT_DATE'].dt.weekday_name
print(df.head())

In [ ]:
le = preprocessing.LabelEncoder()
df['STAT_CAUSE_DESCR'] = le.fit_transform(df['STAT_CAUSE_DESCR'])
df['STATE'] = le.fit_transform(df['STATE'])
df['DISCOVERY_DAY_OF_WEEK'] = le.fit_transform(df['DISCOVERY_DAY_OF_WEEK'])
print(df.head())

In [ ]:
df['CONT_DAY_OF_WEEK']=df['CONT_DAY_OF_WEEK'].fillna("Unknown")
df['CONT_DAY']=df['CONT_DAY'].fillna("0")
df['CONT_MONTH']=df['CONT_MONTH'].fillna("0")
df['CONT_TIME']=df['CONT_TIME'].fillna("0")
df['DISCOVERY_TIME']=df['DISCOVERY_TIME'].fillna("0")

df['CONT_DAY_OF_WEEK'] = le.fit_transform(df['CONT_DAY_OF_WEEK'])
df['FIRE_SIZE_CLASS'] = le.fit_transform(df['FIRE_SIZE_CLASS'])
print(df.head())

In [ ]:
df['CONT_MONTH']=df['CONT_MONTH'].astype('Float64')
df['CONT_DAY']=df['CONT_DAY'].astype('Float64')
#df['CONT_TIME']=df['CONT_TIME'].astype('Float64')
#df['DISCOVERY_DATE']=df['DISCOVERY_DATE'].astype('Float64')

In [ ]:
for item in df:
    print(item)
    print(df[item].dtype)

In [ ]:
labels=df['FIRE_SIZE_CLASS']
labels.head()

In [ ]:
logits=df.drop(['FIRE_SIZE','FIRE_SIZE_CLASS','DISCOVERY_DATE','CONT_DATE','STATE','CONT_TIME','DISCOVERY_TIME'],axis=1)
logits.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(logits,labels,test_size=0.2)

In [ ]:
reg = LinearRegression().fit(x_train, y_train)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
x_test.first

In [ ]:
#reg.predict(x_test[1])

In [ ]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.optimizers import SGD

In [ ]:
from keras.activations import relu

In [ ]:
labels_cat=keras.utils.to_categorical(labels,7)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(logits,labels_cat,test_size=0.2)

In [ ]:
labels_cat.shape

In [ ]:
model=Sequential()
model.add(BatchNormalization(input_shape=[10]))
model.add(Dense(500,kernel_initializer='truncated_normal'))
model.add(Activation('sigmoid'))
model.add(Dense(300,kernel_initializer='truncated_normal'))
model.add(Activation('sigmoid'))
model.add(Dense(7,kernel_initializer='truncated_normal'))
model.add(Activation('softmax'))

In [ ]:
model.compile(optimizer=SGD(lr=0.01, momentum=0.9,decay=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
model.fit(x_train,y_train,epochs=10,validation_data=(np.array(x_test),np.array(y_test)))

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
a=model.predict(x_test)

Main focus:
Write the algorithm, who joins the wildfire data with the airquality data over the closest latitude and longitude and the closest date

In [ ]:
#just a sketch
aqi=aqi.sort_values(['Date'],axis=0)
lat=df['LATITUDE']
long=df['LONGITUDE']
dis_date=df['DISCOVERY_DATE']
measure_date=aqi['Date']
lat2=aqi['Latitude']
long2=aqi['Longitude']
new_df=pd.DataFrame()#index=range(df.shape[0]))

for i in range(df.shape[0]):
    print(i)
    MinDistance=10000000000000
    MinDistanceCloumn=None

    fitting_locations=[]
    for j in range(aqi.shape[0]): 
        Distance = ((lat.iloc[i]-lat2.iloc[j])**2 + (long.iloc[i]-long2.iloc[j])**2)**0.5  
        Distace=round(Distance,6)

        if Distance < MinDistance:
            MinDistance=Distance
            fitting_locations=[]
            fitting_locations.append(j)
            #print(Distance)
        else:
            if Distance == MinDistance:
                fitting_locations.append(j)
                #print(Distance)
            
    #print(MinDistance)
    #print(fitting_locations)

    for k in fitting_locations:
        if dis_date.iloc[i]<measure_date.astype('datetime64').iloc[k]:
            #print("k gefunden")
            #print(k)
            right_column=aqi.iloc[k]
            #right_column=right_column.swapaxes(0,1)
            #right_column=pd.Series(right_column,index=i)
            #print(right_column)
            #print(df.iloc[i])
            #join together
            break
    new_series=pd.concat([df.iloc[i],right_column])
    #new_series=pd.DataFrame(new_series).transpose()
    #print("new series:")
    #print(new_series)
    #print("_________________________")
    new_df=pd.concat([new_df,new_series],axis=1)
    #print("new df:")
    #print(new_df)
    #print("_________________________")

In [ ]:
new_df=new_df.swapaxes(0,1)
print(new_df)

In [ ]:
new_Series=pd.DataFrame(new_series)
new_Series.head()

In [ ]:
new_Series.transpose()

In [ ]:
new_df.swapaxes(0,1)

In [ ]:
    #new_series=pd.concat([df.iloc[i],right_column])
    new_df=pd.concat([new_df,new_series])

In [ ]:
right_column.transpose()

In [ ]:
right_column

In [ ]:
import pandas as pd


def preprocess_site_list(site_file):
    # Make sure that state, county, and site codes are read as strings
    # instead of numbers
    col_types = {'State Code': str, 'County Code': str, 'Site Number': str}

    sites = pd.read_csv(site_file, dtype=col_types)

    # Columns that can be dropped from the site listing
    cols_to_drop = [
        'Land Use', 'Location Setting', 'Met Site State Code',
        'Met Site County Code', 'Met Site Site Number', 'Met Site Type',
        'Met Site Distance', 'Met Site Direction', 'GMT Offset',
        'Owning Agency', 'Local Site Name', "Address", "Zip Code",
        "State Name", "County Name", "City Name", "CBSA Name", "Tribe Name",
        "Extraction Date"
    ]

    sites = sites.drop(columns=cols_to_drop)

    # Convert site closing dates to datetime64 objects (does not work
    # with read_csv())
    sites['Site Closed Date'] = pd.to_datetime(sites['Site Closed Date'])

    # Get sites that have been closed down before 1992 and drop them
    # from the list
    outdated = sites.loc[
        sites['Site Closed Date'] < pd.to_datetime('01-01-1992')]
    sites = sites.drop(outdated.index)

    # Get the complete site code, add it as a column, and finally
    # use the site code as the index
    sites['site-code'] = sites['State Code'] + '-' + sites[
        'County Code'] + '-' + sites['Site Number']
    sites = sites.set_index('site-code')

    # Divide the data into two Series: one with latitude coordinates and one
    # with longitude coordinates
    sites_lat = sites['Latitude']
    sites_lon = sites['Longitude']

    # Transform the site listings into a dictionary where keys are site
    # codes and values are coordinates}
    sites_lat = sites_lat.to_dict()
    sites_lon = sites_lon.to_dict()

    # Return the two dictionaries with the coordinates
    return sites_lat, sites_lon


def preprocess_aqi(site_file, aqi_file):
    # Read daily AQI measurements from year XXXX
    aqi = pd.read_csv(aqi_file)

    # Drop unnecessary columns
    cols_to_drop = [
        'State Name', 'State Code', 'county Name', 'County Code',
        'Number of Sites Reporting'
    ]
    aqi.drop(columns=cols_to_drop, inplace=True)

    # Get coordinate information for measurement sites
    lat_coords, lon_coords = preprocess_site_list(site_file)

    # Add column containing the site information (including location
    # coordinates) to each AQI measurement
    aqi['Latitude'] = aqi['Defining Site'].map(lat_coords)
    aqi['Longitude'] = aqi['Defining Site'].map(lon_coords)

    return aqi



aqi = preprocess_aqi('aqs_sites.csv', 'daily_aqi_by_county_2004.csv')
print(aqi.head())



In [ ]:
preprocess_aqi("aqs_sites.csv","daily_aqi_by_county_2015.csv")

In [ ]:
import xgboost as xgb

In [ ]:
clf = xgb.XGBModel()

clf.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        eval_metric='logloss',
        verbose=True)

ypred = clf.predict(x_test)
print(ypred)

In [ ]:
df['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df_arson = df[df['STAT_CAUSE_DESCR']=='Arson']
df_arson['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
def plot_corr(df,size=10):
    corr = df.corr()  #the default method is pearson
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr,cmap=plt.cm.Oranges)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)    
    plt.show()        
plot_corr(df)

In [ ]:
dfplot=df

In [ ]:
dfplot=dfplot.sort_index(by=['FIRE_SIZE'],axis=0, ascending=False)

In [ ]:
dflol=dfplot[:500]

In [ ]:
dflol.plot(kind='scatter',x='LONGITUDE',y='LATITUDE',color='coral',alpha=0.3)
plt.show()

In [ ]:
df.head()

In [ ]:
df_lightning = df#[df['STAT_CAUSE_DESCR']=='Lightning']
df_lightning['DISCOVERY_DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df['DISCOVERY_DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df['STATE'].value_counts().head(n=10).plot(kind='barh',color='coral')
plt.show()

In [ ]:
df_CA = df[df['STATE']=='CA']
df_CA['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral',title='causes of fires for CA')
plt.show()